# Workshop 1: JOINS und Subqueries

* Schwierigkeit: einfach
* Prüfungsrelevant: ja
* Zeitaufwand: 45min

Die Lösungen finden sich am Ende dieses Notebooks. Bitte nicht spicken!

## Datenbank-Schema

Hier nochmals das Datenbankdiagram als Referenz.

![Schema](./img/sqlite-sample-database-diagram-color.png)

In [ ]:
# Hilfsfunktionen laden und ...
from util.sqlite_util import *

# ... Test-Datenbank öffnen
path = get_connection_path("chinook.sqlite")
connection = create_connection(path)

##  JOINS über mehrere Tabellen
Joins können auch mehrere Tabellen umfassen, indem einfach mehrere JOINS hintereinander aufgelistet werden. 


In [ ]:
# Beispiel mit vier Tabellen: 
# Zeige alle Tracks welche 'Isabelle Mercier' gekauft hat.
stmt = """SELECT tr.name
FROM customers cu
    INNER JOIN invoices iv USING(CustomerId)
    INNER JOIN invoice_items ii USING(InvoiceId)
    INNER JOIN tracks tr USING(TrackId)
    WHERE cu.firstname = "Isabelle" AND cu.lastname = "Mercier"
        ;"""
cur = execute_query(connection, stmt)
print_results(cur)


['Name']
('Prá Dizer Adeus',)
('Família',)
('Act IV, Symphony',)
('Music for the Funeral of Queen Mary: VI. "Thou Knowest, Lord, the Secrets of Our Hearts"',)
('Partita in E Major, BWV 1006A: I. Prelude',)
('Sing Joyfully',)
('Long As I Can See The Light',)
('Sweet Hitch-Hiker',)
('Born On The Bayou',)
('Night Time Is The Right Time',)


### Aufgabe 9 : Joins über mehrere Tabellen

In [ ]:
# Aufgabe #9: Für 10 Tracks LIMIT 10), zeige deren Namen, den Albumtitel und den Namen des Artisten.


### Aufgabe 9+ (Zusatzaufgabe)

In [ ]:
#Aufgabe 9+: Finde die Anzahl Tracks mit Mediatype "MPEG audio file", die pro Person/Customer gekauft wurden.


## Self-JOINS

Bei self-joins wird eine Tabelle `table` mit sich selber verknüpft. 
![Self-Join](./img/self-join.png)

Damit werden Hierarchien abgebildet oder Einträge miteinander verglichen.

```sql
SELECT * FROM table t1 
   INNER JOIN table t2  
   ON t1.id = t2.other_id;
```


In [ ]:
#Im folgenden Beispiel wird eine hierarchische Beziehung zwischen Employee und Manager abgefragt.
stmt = """
SELECT m.firstname || ' ' || m.lastname AS 'Manager',
       e.firstname || ' ' || e.lastname AS 'Direct report' 
    FROM employees e
    INNER JOIN employees m ON m.employeeid = e.reportsto
    ORDER BY manager;"""
cur = execute_query(connection, stmt)
print_results(cur)

['Manager', 'Direct report']
('Andrew Adams', 'Nancy Edwards')
('Andrew Adams', 'Michael Mitchell')
('Michael Mitchell', 'Robert King')
('Michael Mitchell', 'Laura Callahan')
('Nancy Edwards', 'Jane Peacock')
('Nancy Edwards', 'Margaret Park')
('Nancy Edwards', 'Steve Johnson')


### Aufgabe 10: Self-join

In [ ]:
# Aufgabe 10: Folgende query gibt uns alle Personen nach Stadt geordnet.
# Der ||-Operator verbindet zwei Strings. 
stmt = """SELECT DISTINCT
	e1.city,
	e1.firstName || ' ' || e1.lastname AS fullname
FROM
	employees e1
ORDER BY
	e1.city;
"""
cur = execute_query(connection, stmt)
print_results(cur)

# Wie drucken wir nun nur diejenigen Städte, in welchen mindestens zwei Personen wohnen?
# Tipp: self-join verwenden und sicherstellen, dass die Stadt gleich ist, die Person aber nicht.


['City', 'fullname']
('Calgary', 'Nancy Edwards')
('Calgary', 'Jane Peacock')
('Calgary', 'Margaret Park')
('Calgary', 'Steve Johnson')
('Calgary', 'Michael Mitchell')
('Edmonton', 'Andrew Adams')
('Lethbridge', 'Robert King')
('Lethbridge', 'Laura Callahan')


## Subqueries

```sql
SELECT column_1 
  FROM table_1 
  WHERE column_1 = 
  (
    SELECT column_2 
    FROM table_2 
  );
```

* Subqueries sind erlaubt nach `SELECT`, `FROM` , `HERE` und `JOIN`.
* Subqueries geben oftmals einen einzigen Wert zurück (Ausnahme: IN-Operator)




In [ ]:
# Beispiel 1: finde das Album mit title='Let There Be Rock' und zeige alle tracks in diesem Album. 
# PS: Diese Query könnte auch mit einem Join implementiert werden. Ausprobieren ist freiwillig.
stmt = """
SELECT trackid, name, albumid 
  FROM tracks 
  WHERE albumid = 
  ( 
    SELECT albumid 
      FROM albums 
      WHERE title = "Let There Be Rock" 
  );"""
cur = execute_query(connection, stmt)
print_results(cur)

['TrackId', 'Name', 'AlbumId']
(15, 'Go Down', 4)
(16, 'Dog Eat Dog', 4)
(17, 'Let There Be Rock', 4)
(18, 'Bad Boy Boogie', 4)
(19, 'Problem Child', 4)
(20, 'Overdose', 4)
(21, "Hell Ain't A Bad Place To Be", 4)
(22, 'Whole Lotta Rosie', 4)


In [ ]:
# Beispiel 2: finde alle Kunden mit einem oder einer Supportverantwortlichen in 'Canada'. 
stmt="""
SELECT customerid, firstname, lastname 
  FROM customers
  WHERE supportrepid IN 
  ( 
    SELECT employeeid FROM employees 
    WHERE country = "Canada" 
  ); 
"""
cur = execute_query(connection, stmt)
print_results(cur)

['CustomerId', 'FirstName', 'LastName']
(1, 'Luís', 'Gonçalves')
(2, 'Leonie', 'Köhler')
(3, 'François', 'Tremblay')
(4, 'Bjørn', 'Hansen')
(5, 'František', 'Wichterlová')
(6, 'Helena', 'Holý')
(7, 'Astrid', 'Gruber')
(8, 'Daan', 'Peeters')
(9, 'Kara', 'Nielsen')
(10, 'Eduardo', 'Martins')


In [ ]:
# Beispiel 3: Finde alle Kunden mit einer Rechnung. 
stmt="""
SELECT
    FirstName || " " || LastName name, Company
FROM
    Customers c
WHERE
    EXISTS (
        SELECT 
            1 
        FROM 
            Invoices
        WHERE 
            CustomerId = c.CustomerId
    )
ORDER BY
    name; 
"""
cur = execute_query(connection, stmt)
print_results(cur)

['name', 'Company']
('Aaron Mitchell', None)
('Alexandre Rocha', 'Banco do Brasil S.A.')
('Astrid Gruber', None)
('Bjørn Hansen', None)
('Camille Bernard', None)
('Daan Peeters', None)
('Dan Miller', None)
('Diego Gutiérrez', None)
('Dominique Lefebvre', None)
('Eduardo Martins', 'Woodstock Discos')


In [ ]:
# Beispiel 4: Finde die durchschnittliche Album-Grösse in MB. Dazu müssen zuerst alle Tracks zusammengezählt werden.
stmt="""
SELECT 
  ROUND(AVG(album.size) / 1024 / 1024)
FROM 
  ( 
    SELECT SUM(bytes) size 
      FROM tracks 
      GROUP BY albumid
  ) AS album;
"""
cur = execute_query(connection, stmt)
print_results(cur)

['ROUND(AVG(album.size) / 1024 / 1024)']
(323.0,)


### Aufgabe 11: Subqueries mit IN

In [ ]:
#Aufgabe 11: Folgende Query listet alle Genres vom Typ "Alternative" 
stmt = """
SELECT genreid, name 
  FROM genres WHERE name LIKE "alt%";
"""
# Augabe: Zähle die Anzahl Tracks die zum Genre "Alternative" gehören. 


### Aufgabe 12: Subqueries mit [NOT] EXISTS

In [ ]:
# Aufgabe 12a: Suche die Anzahl Artists die in mindestens ein Album zugewiesen haben.
# Aufgabe 12b: Suche die Anzahl Artists die keine Album zugewiesen haben.


## Lösungen

In [ ]:
# Aufgabe #9: Für 10 Tracks LIMIT 10), zeige deren Namen, den Albumtitel und den Namen des Artisten.
stmt = """SELECT tr.name, al.Title, ar.Name
FROM tracks tr
    INNER JOIN albums al USING(AlbumId)
    INNER JOIN artists ar USING(ArtistId)
        LIMIT 10;"""
cur = execute_query(connection, stmt)
print_results(cur)

['Name', 'Title', 'Name']
('For Those About To Rock (We Salute You)', 'For Those About To Rock We Salute You', 'AC/DC')
('Put The Finger On You', 'For Those About To Rock We Salute You', 'AC/DC')
("Let's Get It Up", 'For Those About To Rock We Salute You', 'AC/DC')
('Inject The Venom', 'For Those About To Rock We Salute You', 'AC/DC')
('Snowballed', 'For Those About To Rock We Salute You', 'AC/DC')
('Evil Walks', 'For Those About To Rock We Salute You', 'AC/DC')
('C.O.D.', 'For Those About To Rock We Salute You', 'AC/DC')
('Breaking The Rules', 'For Those About To Rock We Salute You', 'AC/DC')
('Night Of The Long Knives', 'For Those About To Rock We Salute You', 'AC/DC')
('Spellbound', 'For Those About To Rock We Salute You', 'AC/DC')


In [ ]:
#Aufgabe 9+: Finde die Anzahl Tracks mit Mediatype "MPEG audio file", die pro Person/Customer gekauft wurden.
stmt = """SELECT cu.firstname, cu.lastname, count(tr.name)
FROM customers cu
    INNER JOIN invoices iv USING(CustomerId)
    INNER JOIN invoice_items ii USING(InvoiceId)
    INNER JOIN tracks tr USING(TrackId)
    INNER JOIN media_types mt USING(MediaTypeId)
    WHERE mt.name = "MPEG audio file"
    GROUP BY  cu.customerid
        ;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(len(rows))
print_results(cur)

59
['FirstName', 'LastName', 'count(tr.name)']


In [ ]:
# Aufgabe 10: Folgende query gibt uns alle Personen nach Stadt geordnet.
# Der ||-Operator verbindet zwei Strings. 
stmt = """SELECT DISTINCT
	e1.city,
	e1.firstName || ' ' || e1.lastname AS fullname
FROM
	employees e1
ORDER BY
	e1.city;
"""
cur = execute_query(connection, stmt)
print_results(cur)

# Wie drucken wir nun nur diejenigen Städte, in welchen mindestens zwei Personen wohnen?
# Tipp: self-join verwenden und sicherstellen, dass die Stadt gleich ist, die Person aber nicht.
stmt="""SELECT DISTINCT
	e1.city,
	e1.firstName || ' ' || e1.lastname AS fullname
FROM
	employees e1
INNER JOIN employees e2 ON e2.city = e1.city 
   AND (e1.firstname <> e2.firstname AND e1.lastname <> e2.lastname)
ORDER BY
	e1.city;
"""
cur = execute_query(connection, stmt)
print("----")
print_results(cur)

['City', 'fullname']
('Calgary', 'Nancy Edwards')
('Calgary', 'Jane Peacock')
('Calgary', 'Margaret Park')
('Calgary', 'Steve Johnson')
('Calgary', 'Michael Mitchell')
('Edmonton', 'Andrew Adams')
('Lethbridge', 'Robert King')
('Lethbridge', 'Laura Callahan')
----
['City', 'fullname']
('Calgary', 'Nancy Edwards')
('Calgary', 'Jane Peacock')
('Calgary', 'Margaret Park')
('Calgary', 'Steve Johnson')
('Calgary', 'Michael Mitchell')
('Lethbridge', 'Robert King')
('Lethbridge', 'Laura Callahan')


In [ ]:
#Aufgabe 11: Folgende Query listet alle Genres vom Typ "Alternative" 
stmt = """
SELECT genreid, name 
  FROM genres WHERE name LIKE "alt%";
"""
# Augabe: Zähle die Anzahl Tracks die zum Genre "Alternative" gehören. 
stmt = """
SELECT COUNT(*)
    FROM tracks t
WHERE t.genreid IN
 (SELECT genreid 
  FROM genres WHERE name LIKE "alt%");"""
cur = execute_query(connection, stmt)
print_results(cur)

['COUNT(*)']
(372,)


In [ ]:
# Aufgabe 12a: Suche die Anzahl Artists die in mindestens ein Album zugewiesen haben.
# Aufgabe 12b: Suche die Anzahl Artists die keine Album zugewiesen haben.
stmt="""
SELECT count(*) FROM 
    artists a
WHERE EXISTS (
    SELECT 1 FROM albums
    WHERE a.artistid = artistid
);
"""
cur = execute_query(connection, stmt)
print_results(cur)

stmt="""
SELECT count(*) FROM 
    artists a
WHERE NOT EXISTS (
    SELECT 1 FROM albums
    WHERE a.artistid = artistid
);
"""
cur = execute_query(connection, stmt)
print_results(cur)

['count(*)']
(204,)
['count(*)']
(70,)
